In [1]:
import pandas as pd 

#For debugging within For Loops
from IPython.display import display

#calculate the number of years we gots.
starting_year = 1990

#last_year = 2019, so 29 years 
num_of_years = 29

In [2]:
#Preparing the final DF that will be the aggregate of every year's data
final_df = pd.DataFrame()

In [9]:
#Loop through all the years
for i in range(num_of_years):
    display(i)
    #Make the URL
    url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year=' + str(starting_year)
    #Read the tables and make a DF from it
    tables = pd.read_html(url)
    df = tables[1]
    
    #fix the header issues
    df.columns = df.iloc[0]
    df = df[1:]

#     display(df.head())
    #Reformat the date
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')
    df['Date'] = df['Date'].dt.strftime('%m/%Y')
    
    
    #Turn daily data into monthly mean data
    df_grouped_by_months = df.groupby('Date')
    df_monthly = df_grouped_by_months.mean()
    
    #Turn monthly data into rolling mean of every 3 months data
    rolling_df = df_grouped_by_months.rolling(3).mean()
    
    #Turn rolling mean into Quarterly Data
    #We just delete every month except for march june september december and we'll have quarterly data
    
    #Reseting index real quick so we can delete rows by index
    rolling_df = rolling_df.reset_index()
#     quarterly_df = rolling_df.drop(rolling_df[test.x > 0].index)
    
    #We keep rows of index 2,5,8,11 (since index starts at 0)
    quarterly_df = rolling_df.drop([0,1,3,4,6,7,9,10])
    final_df = pd.concat([final_df,quarterly_df], axis=0, ignore_index=True)
    starting_year += 1

0

/Users/minglei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/minglei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


DataError: No numeric types to aggregate

In [ ]:
#Lets take a look at where we are...
final_df.tail()

In [ ]:
#writting what we found to CSV. 
final_df.to_csv("resources/yield.csv", index=False, date_format='%m/%Y')



In [ ]:
final_df.columns